In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [15]:
data.printSchema()
data.columns


root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [10]:
from pyspark.ml.linalg import Vectors

In [11]:
from pyspark.ml.feature import VectorAssembler

In [16]:
Asem = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website','Length of Membership']
                       , outputCol='features')

In [21]:
output = Asem.transform(data)
output.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [19]:
final_data = output.select('features','Yearly Amount Spent')

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [24]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [25]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                357|
|   mean| 498.30216757202214|
| stddev|   78.3084699689515|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                143|
|   mean| 501.84017696561904|
| stddev|  81.99876870391341|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [26]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [27]:
lr_model = lr.fit(train_data)

In [28]:
test_results = lr_model.evaluate(test_data)

In [29]:
test_results.rootMeanSquaredError

10.697769454919278

In [36]:
test_results.predictions.select('prediction').show()


+------------------+
|        prediction|
+------------------+
|331.42148622360014|
|387.82675440662024|
| 565.1666797777264|
| 535.0370716529158|
|482.04215868074243|
|495.18737294360744|
|449.73479929281416|
| 559.2551594335141|
|380.32348873754313|
|468.47906075252376|
|495.26192467555074|
| 501.3224414418073|
| 448.9757667350216|
|  449.187573737667|
|  558.334888895093|
| 398.1805801159842|
|  491.320740890397|
|380.09608675339496|
| 663.6856192944363|
| 463.7478714258077|
+------------------+
only showing top 20 rows



In [45]:
test_results.r2
test_results.rootMeanSquaredError

10.697769454919278

In [40]:
unlabeled_data = train_data.select('features')

In [42]:
prediction = lr_model.transform(unlabeled_data)

In [43]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|398.69556626830536|
|[30.4925366965402...|287.27172139191316|
|[30.5743636841713...| 442.3424898033445|
|[30.7377203726281...| 451.3834116280641|
|[30.8162006488763...| 282.4440848616198|
|[30.8364326747734...| 471.9868383982016|
|[30.8794843441274...| 494.2439926815773|
|[30.9716756438877...| 488.4712503606613|
|[31.0613251567161...| 494.1636493922483|
|[31.0662181616375...|  462.254810426482|
|[31.1239743499119...| 509.0490626920616|
|[31.1695067987115...| 416.8094138048514|
|[31.2606468698795...| 421.8704900467965|
|[31.2681042107507...| 427.5194566636653|
|[31.2834474760581...| 570.4491796582495|
|[31.3091926408918...| 429.8996635667154|
|[31.3123495994443...|444.71770918915786|
|[31.3584771924370...|491.43312803414346|
|[31.3662121671876...|426.99489300580944|
|[31.3895854806643...| 409.5121870304886|
+--------------------+------------